In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [12]:
#x_train.filter(lambda v: v==v, x)

In [13]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [14]:
np.shape(x_train)

(250000, 30)

In [15]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [16]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x)
    return (y_pred == y).sum()/len(y_pred)

In [17]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [18]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [19]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [20]:
np.shape(np.asarray([1]*30))

(30,)

In [21]:
initial_w = np.asarray([1]*30)
max_iters = 30
gamma = 0.7

In [22]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)

/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/loss.py:5: RuntimeWarning: overflow encountered in square
  return np.mean(e**2)/2.


In [23]:
compare_prediction(w_sgd, x_train, y_train)

0.62795599999999996

### Ridge Regression

In [24]:
from build_polynomial import build_poly

def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    tr_poly = build_poly(x_tr, degree)
    te_poly = build_poly(x_te, degree)
    print(np.shape(x_tr))
    print(np.shape(tr_poly))
    #####
    #tr_poly = np.sum(np.split(tr_poly, degree+1, axis = 1), axis=0)
    #te_poly = np.sum(np.split(te_poly, degree+1, axis = 1), axis=0)
    #####
    
    loss = 1000
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, tr_poly, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, te_poly, weight)
        
        if(mse_test < loss):
            loss = mse_test
            w = weight
        
    return w, loss

In [63]:
# boucler sur les seeds pour avoir un meileur ? 
seed = 69
degree = 6
split_ratio = 0.9

w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

(225000, 30)
(225000, 210)


In [26]:
np.shape(y_train)

(250000,)

In [27]:
np.shape(w_r)

(210,)

In [28]:
w_s = np.sum(np.split(w_r, degree + 1),axis=0)
np.shape(w_s)

(30,)

In [29]:
np.shape(build_poly(x_train, degree))

(250000, 210)

In [64]:
compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))

0.79982799999999998

### Logistic Regression

In [31]:
def ridge_regression2(y, tx, lamb):
    """implement ridge regression."""
    aI = lamb * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression3(y, tx, lambda_):
    first_term = tx.T.dot(tx)
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T.dot(y)
    w = np.linalg.solve(left, right)
    loss = compute_loss2(y, tx, w)
    return w, loss;

def compute_loss2(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)/2.

### Bias-variance

In [32]:
from implementations import least_squares
from split_data import split_data

def bias_variance_demo():
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambdas = np.logspace(-5, 0, 15)
    
    best_seed = 0
    best_degree = 0
    best_lambda = 0
    min_loss = 1000
    for index_seed, seed in enumerate(seeds):
        np.random.seed(seed)
    
        # split data with a specific seed
        x_tr, y_tr, x_te, y_te = split_data(x_train, y_train, ratio_train, seed)
        
        # bias_variance_decomposition
        for index_deg, degree in enumerate(degrees):
            # form data with polynomial degree
            train_poly = build_poly(x_tr, degree)
            test_poly = build_poly(x_te, degree)
            
            w_train, loss_train = least_squares(y_tr, train_poly)

            # calculate the loss for train and test data
            loss_test = compute_loss(y_te, test_poly, w_train)
            
            if(loss_test < min_loss):
                min_loss = loss_test
                best_seed = seed
                #best_degree = degree
                best_lambda = lambda_
    #return best_seed, best_lambda, min_loss
    return best_seed, best_degree ,min_loss        


In [33]:
#s, d, l = bias_variance_demo()

In [34]:
#l

### Cross-validation

In [35]:
from build_k_indices import build_k_indices
from implementations import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]

    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)

    # ridge regression
    
    weights_train, loss_tr = ridge_regression(y_tr, x_tr, lambda_)

    # calculate the loss for train and test data
    loss_te = compute_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te

In [36]:
def cross_validation_demo():
    seed = 55
    degree = 2
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    min_mean_loss = 1000
    best_lambdas = np.zeros(k_fold)
    # cross validation
    for i_test in range(k_fold):
        min_loss = 1000
        for lambda_ in lambdas:
            loss_tr, loss_te = cross_validation(y_train, x_train, k_indices, i_test, lambda_, degree)
            
            if(loss_te < min_loss):
                min_loss = loss_te
                best_lambdas[i_test] = lambda_
    lambda_mean = np.mean(best_lambdas)
    return lambda_mean, min_mean_loss

In [37]:
bl, mml = cross_validation_demo()

In [38]:
bl

0.25007499999999999

In [39]:
w, l = ridge_regression(y_train, x_train, bl)

In [40]:
compare_prediction(w, x_train, y_train)

0.74440799999999996

### Logistic Regression

In [41]:
max_iters = 100
gamma = 0.01

x_train2 = x_train
y_train2 = y_train.reshape(len(y_train), 1)

print(np.shape(x_train2))
print(np.shape(y_train2))

#tx = np.c_[np.ones((y.shape[0], 1)), x]
w = np.zeros((x_train2.shape[1], 1))

w , loss = logistic_regression(y_train2, x_train2, w, max_iters, gamma)

(250000, 30)
(250000, 1)


In [42]:
print(loss)
np.shape(w)

173286.79514


(30, 1)

In [43]:
#marche pas ... a voir

#compare_prediction(w, x_train, y_train)

In [44]:
def logistic_regression_demo():
    max_iters = 2
    #gamma = 0.01
    gammas = np.arange(0.01,0.5,0.01)
    
    x_train2 = x_train
    y_train2 = y_train.reshape(len(y_train), 1)
    
    w = np.zeros((x_train2.shape[1], 1))
    
    best_w = 0
    best_loss = 1000
    best_gamma = 0
    
    for gamma_ in gammas:
        
        w , loss = logistic_regression(y_train2, x_train2, w, max_iters, gamma_)
        
        #print(gamma_)
        #print(loss)
        
        if(loss < best_loss ):
            best_loss = loss
            best_gamma = gamma_
            best_w = w

    
    return best_w, best_loss


In [45]:
# Problem
w, loss = logistic_regression_demo()

/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/loss.py:11: RuntimeWarning: overflow encountered in exp
  loss = np.log(1 + np.exp(tx @ w)) - (y * (tx @ w))
/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/sigmoid.py:6: RuntimeWarning: overflow encountered in exp
  return (np.exp(t) / (1 + np.exp(t)))
/Users/kevinkappel/Desktop/EPFL/Master1/ML/Isaac/ML_project/project1/template/sigmoid.py:6: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(t) / (1 + np.exp(t)))


In [46]:
w

0